In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
data = pd.read_csv("quotedata.dat", skiprows=2)

#### Set Rate 

In [3]:
from bs4 import BeautifulSoup as bs4
import requests

In [4]:
url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
req = requests.get(url)
soup = bs4(req.text)

In [5]:
# the one month t-bill rate
RATE = float(list(soup.find('tr', attrs=({'class':"oddrow"})))[1].text) / 100

#### Set time-NOW. Leave as '2020-03-23 16:00:00.000000' for proof of concept

In [6]:
# set time-NOW
# NOW = dt.datetime.NOW()
NOW = pd.to_datetime('2020-03-23 16:00:00.000000')

In [7]:
# set nearest date limit
near_date = NOW + dt.timedelta(days=23)

In [8]:
# set farthest date limit
next_date = NOW + dt.timedelta(days=37)

In [9]:
# set minutes in year
MINUTES_IN_YEAR = 525600
MINUTES_IN_DAY = 24 * 60

In [10]:
def filter_quotes(input_data):
    
    # select columns
    input_data = input_data[['Expiration Date', 'Strike', 'Calls', 'Bid', 'Ask', 'Puts', 'Bid.1', 'Ask.1']]
    # filter to between near and next data
    input_data = input_data.loc[(pd.to_datetime(input_data['Expiration Date']) > near_date) & (pd.to_datetime(input_data['Expiration Date']) < next_date)]
    # convert expiration date column to datetime
    input_data['Expiration Date'] = pd.to_datetime(input_data['Expiration Date']).values
    # select only friday expiries
    input_data = input_data.loc[pd.DatetimeIndex(input_data['Expiration Date']).weekday == 4]
    
    return input_data

In [11]:
def filter_non_zero_bids(input_data):
    return input_data.loc[(input_data['Bid'] != 0) & (input_data['Bid.1'] != 0)]

In [12]:
def set_datetime(input_data):
    
    '''had to create new datetime column to replace use of Expiration Date'''
    
    # set time for weekly expiries
    input_data['ExpDT'] = input_data.loc[input_data.Calls.str.contains('W')]['Expiration Date'] + dt.timedelta(hours=16)
    # set time for standard expiries
    input_data.loc[input_data.ExpDT.isna(), 'ExpDT'] = input_data.loc[input_data.ExpDT.isna()]['Expiration Date'] + dt.timedelta(hours=9.5)
    
    return input_data

In [13]:
def set_minutes_remaining(input_data, MINUTES_IN_YEAR):
    
    # set big T to expiration in minutes ... input_data.ExpDT - NOW returns datetime[ns]
    input_data['TTE'] = (((input_data.ExpDT - NOW) / pd.np.timedelta64(1, 'm')) / MINUTES_IN_YEAR).values
    
    return input_data

In [14]:
def set_midpoint(input_data):
    
    input_data['calls_midpoint'] = (input_data['Ask'] + input_data['Bid']) / 2
    input_data['puts_midpoint'] = (input_data['Ask.1'] + input_data['Bid.1']) / 2
    input_data['difference'] = abs(input_data['calls_midpoint'] - input_data['puts_midpoint'])
    
    return input_data

In [15]:
def split_near_next(input_data):
    
    data = input_data.copy()
    data = data.set_index('Expiration Date')
    #  get unique dates, will sort correctly
    dates = [str(i.date()) for i in data.index.unique()]
    #  set nearest date
    near = data.loc[data.index == dates[0]]
    # set next date
    nxt = data.loc[data.index == dates[1]]
    
    return nxt.reset_index(), near.reset_index()

In [16]:
def set_F(input_data, RATE):
    
    strike = input_data.loc[input_data.index == (input_data.loc[input_data.difference == input_data.difference.min()].index[0])]['Strike']
    call = input_data.loc[input_data.index == (input_data.loc[input_data.difference == input_data.difference.min()].index[0])]['calls_midpoint']
    put = input_data.loc[input_data.index == (input_data.loc[input_data.difference == input_data.difference.min()].index[0])]['puts_midpoint']
    tte = input_data.loc[input_data.index == (input_data.loc[input_data.difference == input_data.difference.min()].index[0])]['TTE']
    _F = strike + np.exp(tte * RATE) * (call - put)
    
    return _F.values[0]

In [17]:
def set_K0(input_data, F):
    
    _k0 = input_data.loc[input_data.Strike < F].iloc[-1]['Strike']
    
    return _k0

In [18]:
def set_deltaK(input_data):
    return ((input_data.Strike.shift(-1) - input_data.Strike.shift(1)) / 2).values

In [19]:
def set_Ki_squared(input_data):
    return (input_data.Strike ** 2).values

In [20]:
def set_deltaK_dividedby_KiSqrd(input_data):
    return (input_data.deltaK / input_data.Ki_sqrd).values

In [21]:
def set_eRT(input_data, RATE):
    return np.exp(input_data.TTE.values * RATE)

In [22]:
def set_QKi(input_data, some_strike_K0):
    
    # create single midpoint column, assigning calls midpoint immediately
    input_data['midpoint'] = (input_data.Strike >= some_strike_K0) * input_data.calls_midpoint
    # assign puts midpoint
    input_data.loc[input_data.Strike < some_strike_K0 , 'midpoint'] = input_data.loc[input_data.Strike < some_strike_K0]['puts_midpoint'].values
    # average the call and put midpoint for K0
    input_data.loc[input_data.Strike == some_strike_K0 , 'midpoint'] = np.mean(input_data.loc[input_data.Strike == some_strike_K0]['puts_midpoint'].values + input_data.loc[input_data.Strike == some_strike_K0]['calls_midpoint'].values)
    
    return input_data

In [23]:
def prepare_quotes(input_data, RATE, MINUTES_IN_YEAR):
    
    #  filter for near & next, friday expiries, non-zero bids
    mod = filter_quotes(input_data)
    #  create new expiration date column to display time
    mod = set_datetime(mod)
    #  create time to expiry in minutes
    mod = set_minutes_remaining(mod, MINUTES_IN_YEAR)
    #  set index
    mod = mod.reset_index()
    mod = mod.drop('index', axis=1)
    #  get midpoint
    mod = set_midpoint(mod)
    
    #  split
    nxt, near = split_near_next(mod)
    #  F Forward index level derived from index option prices of near
    F1 = set_F(near, RATE)
    #  F Forward index level derived from index option prices of next
    F2 = set_F(nxt, RATE)
    
    # Set K0 for next
    nxt_K0 = set_K0(nxt, F2)
    # set K0 for near
    near_K0 = set_K0(near, F1)
    
    # set Q(K_i) for near
    near = set_QKi(near, near_K0)
    # set Q(K_i) for next
    nxt = set_QKi(nxt, nxt_K0)
    
    # set delta K for near
    near['deltaK'] = set_deltaK(near)
    # set delta K for next
    nxt['deltaK'] = set_deltaK(nxt)
    
    # set K_i ** 2 for near
    near['Ki_sqrd'] = set_Ki_squared(near)
    # set K_i ** 2 for next
    nxt['Ki_sqrd'] = set_Ki_squared(nxt)
    
    # set deltaK / Ki^2 for near
    near['delK_div_KiSqrd'] = set_deltaK_dividedby_KiSqrd(near)
    # set deltaK / Ki^2 for next
    nxt['delK_div_KiSqrd'] = set_deltaK_dividedby_KiSqrd(nxt)
    
    # e^rt for near
    near['eRT'] = set_eRT(near, RATE)
    # e^rt for next
    nxt['eRT'] = set_eRT(nxt, RATE)
    
    # filter non zero bids
    nxt = filter_non_zero_bids(nxt)
    near = filter_non_zero_bids(near)
    
    return nxt, near

In [24]:
def set_d1(input_data):
    return ((2/input_data.TTE.values[0]) * np.sum(input_data.delK_div_KiSqrd * input_data.eRT * input_data.midpoint))

In [25]:
def set_d2(input_data, F, K0):
    return ((1/input_data.TTE.values[0]) * ((F / K0)  - 1) ** 2)

In [26]:
def set_sigma(d1, d2):
    return d1 - d2

In [27]:
def calc_sigma(input_data, RATE):
    
    _F = set_F(input_data, RATE)
    _K0 = set_K0(input_data, _F)
    _d1 = set_d1(input_data)
    _d2 = set_d2(input_data, _F, _K0)
    sigma = set_sigma(_d1, _d2)
    
    return sigma

In [28]:
def calc_vix(input_data_from_dat_file, RATE, NOW, MINUTES_IN_YEAR):
    
    nxt, near = prepare_quotes(data, RATE, MINUTES_IN_YEAR)
    
    nxt_sigma = calc_sigma(nxt, RATE)
    
    near_sigma = calc_sigma(near, RATE)
    
    # set t1
    t1 = near.TTE.iloc[0]
    # set t2
    t2 = nxt.TTE.iloc[0]
    # number of minutes in year
    n365 = MINUTES_IN_YEAR
    # number of minutes in 30 days
    n30 = 43200
    # number of minutes till settlement for near
    nt1 = (nxt.ExpDT.iloc[0] - NOW).total_seconds() / 60
    # number of minutes till settlement for next
    nt2 = (near.ExpDT.iloc[0] - NOW).total_seconds() / 60
    
    near_weight = t1 * near_sigma * ((nt2 - n30) / (nt2 - nt1))
    next_weight = t2 * nxt_sigma * ((n30 - nt1) / (nt2 - nt1))
    weighted_values = (near_weight + next_weight) * (n365/n30)
    
    vix = 100 * np.sqrt(weighted_values)
    
    return vix

In [29]:
calc_vix(data, RATE, NOW, MINUTES_IN_YEAR)

66.64653488886552